# Setup

Clone YOLOv5 repo, install dependencies and check PyTorch and GPU.

In [ ]:
!git clone https://github.com/mh-salari/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies
from IPython.display import Image, clear_output

clear_output()

print('Instaling YOLOv5 completed')

Instaling YOLOv5 completed


In [ ]:
import torch
clear_output()
print('Setup completed. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup completed. Using torch 1.9.0+cu102 _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


In [ ]:
!nvidia-smi

Wed Sep 15 20:26:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    32W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Prepare dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!unzip -q "/content/drive/MyDrive/AI-ML/datasets/carvansara_yolo.zip" -d "/content"
!unzip -q "/content/drive/MyDrive/AI-ML/datasets/caravansara_yolo_newdata.zip" -d "/content"
!cp -R /content/caravansara_yolo_newdata/* /content/carvansara_yolo


In [ ]:
dataset_yaml_txt= """
# train and val data 
train: /content/carvansara_yolo/train
val: /content/carvansara_yolo/test

# number of classes
nc: 1

# class names
names: ['carvansara',]
"""

with open("data/carvansara.yaml", "w") as f:
    f.write(dataset_yaml_txt)

## Weights & Biases Logging




In [ ]:
!pip install wandb
!wandb login 8547bbc8a42f4383966c0fc9c58a7520d51fa485
clear_output()
print("Weights & Biases Setup completed.")

Weights & Biases Setup completed.


# Train the models

In [ ]:
hyp = """
lr0: 0.001  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.05  # optimizer weight decay 5e-4 #0.0005
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 15.0  # image rotation (+/- deg) 75
translate: 0.1  # image translation (+/- fraction)
scale: 0.45  # image scale (+/- gain)
shear: 0.001  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 0.0 # image mosaic (probability)
mixup: 0.0  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
"""
with open("/content/yolov5/data/n_hyp.scratch.yaml", "w") as f:
    f.write(hyp)

In [ ]:
# !python train.py --epochs 10 --data carvansara.yaml --weights yolov5s.pt --cache --evolve

In [ ]:
name = "scale_invariant_r"

In [ ]:
!python train.py  --img 576 --batch 4 --epochs 30  --single-cls --project "carvansara" --name {name}  --hyp "data/n_hyp.scratch.yaml" --data carvansara.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=carvansara.yaml, hyp=data/n_hyp.scratch.yaml, epochs=30, batch_size=4, imgsz=576, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=True, adam=False, sync_bn=False, workers=8, project=carvansara, entity=None, name=scale_invariant_r, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=100
github: up to date with https://github.com/mh-salari/yolov5 ✅
YOLOv5 🚀 v5.0-436-g4f1fc32 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

hyperparameters: lr0=0.001, lrf=0.2, momentum=0.937, weight_decay=0.05, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=15.0, translate=0.1, s

## test


In [ ]:
!python val.py  --project "carvansara" --weights carvansara/{name}/weights/best.pt --data carvansara.yaml --task val

val: data=./data/carvansara.yaml, weights=['carvansara/scale_invariant_r/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=carvansara, name=exp, exist_ok=False, half=False
YOLOv5 🚀 v5.0-436-g4f1fc32 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs
val: Scanning '/content/carvansara_yolo/test/labels.cache' images and labels... 362 found, 0 missing, 300 empty, 0 corrupted: 100% 362/362 [00:00<?, ?it/s]
val: Scanning '/content/carvansara_yolo/test/labels.cache' images and labels... 362 found, 0 missing, 300 empty, 0 corrupted: 100% 362/362 [00:00<?, ?it/s]

               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:   8% 1/12 [00:02<00:25,  2.31s/it]
               Class     Images     Labels         

In [ ]:
%ls

carvansara/      Dockerfile  models/            train.py        wandb/
CONTRIBUTING.md  export.py   __pycache__/       tutorial.ipynb  yolov5s.pt
data/            hubconf.py  README.md          utils/
detect.py        LICENSE     requirements.txt*  val.py


# test model

In [ ]:
!zip -r /content/{name}.zip /content/yolov5/carvansara/{name} 

  adding: content/yolov5/carvansara/scale_invariant_r/ (stored 0%)
  adding: content/yolov5/carvansara/scale_invariant_r/train_batch1.jpg (deflated 10%)
  adding: content/yolov5/carvansara/scale_invariant_r/results.csv (deflated 83%)
  adding: content/yolov5/carvansara/scale_invariant_r/confusion_matrix.png (deflated 37%)
  adding: content/yolov5/carvansara/scale_invariant_r/val_batch1_pred.jpg (deflated 10%)
  adding: content/yolov5/carvansara/scale_invariant_r/val_batch0_pred.jpg (deflated 9%)
  adding: content/yolov5/carvansara/scale_invariant_r/opt.yaml (deflated 45%)
  adding: content/yolov5/carvansara/scale_invariant_r/PR_curve.png (deflated 27%)
  adding: content/yolov5/carvansara/scale_invariant_r/weights/ (stored 0%)
  adding: content/yolov5/carvansara/scale_invariant_r/weights/last.pt (deflated 8%)
  adding: content/yolov5/carvansara/scale_invariant_r/weights/best.pt (deflated 8%)
  adding: content/yolov5/carvansara/scale_invariant_r/val_batch0_labels.jpg (deflated 9%)
  addi

In [ ]:
# from google.colab import files
# files.download("/content/yolov5s_448_200_t0.5_s0.05.zip")

In [ ]:
!cp /content/{name}.zip "/content/drive/MyDrive/AI-ML"